Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
import pandas as pd

bike_df = pd.read_csv('bike_stations.csv')
yelp_df = pd.read_csv('new_yelp_df.csv')

# join bike and yelp dataframes
bike_yelp_df = pd.merge(bike_df, yelp_df, how='left', on='station_id')

# save to csv
bike_yelp_df.to_csv('bike_yelp_df.csv', index=False)

# rename column 
bike_yelp_df.rename(columns={'Name':'Restaurant'}, inplace=True)
bike_yelp_df.rename(columns={'latitude':'station_latitude'}, inplace=True)
bike_yelp_df.rename(columns={'longitude':'station_longitude'}, inplace=True)

bike_yelp_df

In [73]:
# check for null values
null_restaurant = bike_yelp_df[bike_yelp_df['Restaurant'].isnull()]
null_rating = bike_yelp_df[bike_yelp_df['Rating'].isnull()]
null_stationid = bike_yelp_df[bike_yelp_df['station_id'].isnull()]
null_address = bike_yelp_df[bike_yelp_df['Address'].isnull()]

null_restaurant
null_rating
null_stationid
null_address

# drop null values - 1 restaurant was missing an address
bike_yelp_df = bike_yelp_df.dropna()

In [74]:
bike_yelp_df.to_csv('bike_yelp_df.csv', index=False)

Duplicates are bound to happen since we are looking at all restaurants that are 1000m away from each bike station, there are restaurants that are going to be 1000m away from multiple bike stations. 

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# histogram of free bike spots
bike_yelp_df['free_bikes'].hist() # most stations have 0-5 free bikes

# histogram of empty bike slots
bike_yelp_df['empty_slots'].hist() # most stations have 5-15 empty slots

When we compare the free bikes and the empty slots, you get a good idea of which areas in the city are busier. However, there are two different ways that we can interpret this data. On one hand, if there are a lot of bikes at a station, it's safe to assume that not many people are renting bikes in that area. It is also safe to assume that the station may be full because many people have already rented their bikes at a different station to get to that location and have returned their bikes. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
import sqlite3
from sqlite3 import Error

df = pd.read_csv('bike_yelp_df.csv')

def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

connection = create_connection("bike_yelp_df.db")

# turn dataframe into sql table
df.to_sql('bike_yelp_df', connection, if_exists='replace', index=False)

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        column_names = [description[0] for description in cursor.description]
        df = pd.DataFrame(result, columns=column_names)
        display(df) # display works better than print in jupyter notebook
    except Error as e:
        print(f"The error '{e}' occurred")
    
# put results in database
sql_query = "SELECT * from bike_yelp_df"
rows = execute_query(connection, sql_query)


Look at the data before and after the join to validate your data.

In [ ]:
bike = pd.read_csv('bike_stations.csv')
yelp = pd.read_csv('new_yelp_df.csv')

bike_yelp = pd.read_csv('bike_yelp_df.csv')

bike
yelp
bike_yelp